<a href="https://colab.research.google.com/github/dariiazorii/OID/blob/main/lab1_oid/LAB1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Перезапуск та встановлення Spark 3.3.4
print("1/4: Встановлення Spark 3.3.4...")
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.3.4/spark-3.3.4-bin-hadoop3.tgz
!tar xf spark-3.3.4-bin-hadoop3.tgz
!pip install -q findspark pyspark

# 2. Налаштування середовища (Критично: зміна шляху SPARK_HOME)
print("2/4: Налаштування змінних середовища...")
import os
import findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.4-bin-hadoop3" # Зміна шляху!
findspark.init()


1/4: Встановлення Spark 3.3.4...
^C
^C
ERROR: Operation cancelled by user
2/4: Налаштування змінних середовища...


In [ ]:
# 3. Монтування Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


In [ ]:
# 4. Створення SparkSession та робота
print("4/4: Створення SparkSession та перетворення...")
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("LAB_1") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()


csv_file = "/content/drive/MyDrive/Colab Notebooks/oid/2019-Oct.csv"
parquet_output = "/content/2019_Oct_parquet"

df = spark.read.csv(csv_file, header=True, inferSchema=True)
df.write.parquet(parquet_output, mode="overwrite")
print("Перетворення CSV у Parquet успішно завершено.")
print("PySpark встановлено та налаштовано. Можна створювати SparkSession.")

4/4: Створення SparkSession та перетворення...
Перетворення CSV у Parquet успішно завершено.
PySpark встановлено та налаштовано. Можна створювати SparkSession.


In [ ]:

gzip_parquet_output = "/content/2019_Oct_parquet_gzip"

print(f"Створення Parquet з Gzip стисненням до: {gzip_parquet_output}...")


df.write.format("parquet") \
    .mode("overwrite") \
    .option("compression", "gzip") \
    .save(gzip_parquet_output)

print(" Gzip-стиснення завершено.")

Створення Parquet з Gzip стисненням до: /content/2019_Oct_parquet_gzip...
 Gzip-стиснення завершено.


In [ ]:
!du -sh "/content/drive/MyDrive/Colab Notebooks/oid/2019-Oct.csv"
!du -sh "/content/2019_Oct_parquet"
!du -sh "{gzip_parquet_output}"

5.3G	/content/drive/MyDrive/Colab Notebooks/oid/2019-Oct.csv
1.4G	/content/2019_Oct_parquet
926M	/content/2019_Oct_parquet_gzip


In [ ]:

import time

csv_path = "/content/drive/MyDrive/Colab Notebooks/oid/2019-Oct.csv"
snappy_path = "/content/2019_Oct_parquet"
gzip_path = "/content/2019_Oct_parquet_gzip"

def run_and_time_count(path, format_name):


    start_time = time.time()


    if "csv" in path:

        df = spark.read.csv(path, header=True, inferSchema=True)
    else:

        df = spark.read.parquet(path)


    count = df.count()

    end_time = time.time()
    duration = end_time - start_time
    print(f"  {format_name} (Кількість рядків: {count}): {duration:.2f} сек.")
    return duration

print("--- Порівняння продуктивності: Підрахунок рядків ---")

# 1. CSV (Оригінал)
time_csv = run_and_time_count(csv_path, "1. CSV (Оригінал)")

# 2. Parquet (Snappy)
time_snappy = run_and_time_count(snappy_path, "2. Parquet (Snappy)")

# 3. Parquet (Gzip)
time_gzip = run_and_time_count(gzip_path, "3. Parquet (Gzip)")

print("\n--- Висновок ---")
print(f"Parquet  у {time_csv / time_snappy:.1f} рази швидше, ніж CSV.")
print(f"Parquet (Gzip) у {time_csv / time_gzip:.1f} рази швидше, ніж CSV.")
print(f"Parquet (Gzip) у {time_snappy / time_gzip:.1f} рази швидше, ніж Parquet (Snappy).")

spark.stop()

--- Порівняння продуктивності: Підрахунок рядків ---
  1. CSV (Оригінал) (Кількість рядків: 42448764): 297.81 сек.
  2. Parquet (Snappy) (Кількість рядків: 42448764): 1.17 сек.
  3. Parquet (Gzip) (Кількість рядків: 42448764): 0.68 сек.

--- Висновок ---
Parquet (Snappy) у 254.1 рази швидше, ніж CSV.
Parquet (Gzip) у 437.3 рази швидше, ніж CSV.
Parquet (Gzip) у 1.7 рази швидше, ніж Parquet (Snappy).
